# Elektron WebSocket API MRN Example with Python

## Machine Readable News Overview

Refinitiv Machine Readable News (MRN) is an advanced service for automating the consumption and systematic analysis of news. It delivers deep historical news archives, ultra-low latency structured news and news analytics directly to your applications. This enables algorithms to exploit the power of news to seize opportunities, capitalize on market inefficiencies and manage event risk.

### MRN Data model
MRN is published over Elektron using an Open Message Model (OMM) envelope in News Text Analytics domain messages. The Real-time News content set is made available over MRN_STORY RIC. The content data is contained in a FRAGMENT field that has been compressed, and potentially fragmented across multiple messages, in order to reduce bandwidth and message size.

A FRAGMENT field has a different data type based on a connection type:
* RSSL connection (ESDK [C++](https://developers.refinitiv.com/elektron/elektron-sdk-cc)/[Java](https://developers.refinitiv.com/elektron/elektron-sdk-java)): BUFFER type
* WebSocket connection: Base64 ascii string

The data goes through the following series of transformations:

1. The core content data is a UTF-8 JSON string
2. This JSON string is compressed using gzip
3. The compressed JSON is split into a number of fragments (BUFFER or Base64 ascii string) which each fit into a single update message
4. The data fragments are added to an update message as the FRAGMENT field value in a FieldList envelope

Therefore, in order to parse the core content data, the application will need to reverse this process. The WebSocket application also need to convert a received Base64 string in a FRAGMENT field to bytes data before further process this field.

Five fields, as well as the RIC itself, are necessary to determine whether the entire item has been received in its various fragments and how to concatenate the fragments to reconstruct the item:
* MRN_SRC: identifier of the scoring/processing system that published the FRAGMENT
* GUID: a globally unique identifier for the data item. All messages for this data item will have the same GUID values.
* FRAGMENT: compressed data item fragment, itself
* TOT_SIZE: total size in bytes of the fragmented data
* FRAG_NUM: sequence number of fragments within a data item. This is set to 1 for the first fragment of each item published and is incremented for each subsequent fragment for the same item.

A single MRN data item publication is uniquely identified by the combination of RIC, MRN_SRC, and GUID.

#### Fragmentation
For a given RIC-MRN_SRC-GUID combination, when a data item requires only a single message, then TOT_SIZE will equal the number of bytes in the FRAGMENT and FRAG_NUM will be 1.

When multiple messages are required, then the data item can be deemed as fully received once the sum of the number of bytes of each FRAGMENT equals TOT_SUM. The consumer will also observe that all FRAG_NUM range from 1 to the number of the fragment, with no intermediate integers skipped. In other words, a data item transmitted over three messages will contain FRAG_NUM values of 1, 2 and 3.

#### Compression
The FRAGMENT field is compressed with gzip compression, thus requiring the consumer to decompress to reveal the JSON plain-text data in that FID.

When an MRN data item is sent in multiple messages, all the messages must be received and their FRAGMENTs concatenated before being decompressed. In other words, the FRAGMENTs should not be decompressed independently of each other.

The decompressed output is encoded in UTF-8 and formatted as JSON.


Please see a full documentation of this example application in [this article](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).

If you are not familiar with MRN concept, please visit the following resources which will give you a full explanation of the MRN data model and implementation logic:
* [Webinar Recording: Introduction to Machine Readable News](https://developers.refinitiv.com/news#news-accordion-nid-12045)
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).

In [1]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install requests and websocket-client packages in a current Jupyter kernal\n

import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [21]:

import time
import getopt
import socket
import json
import websocket
import threading
from threading import Thread, Event
import base64
import zlib

In [22]:
# TREP connection variables

hostname = '127.0.0.1'
port = '15000'
user = 'root'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [24]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False
_news_envelopes = []

# keeps decompress news JSON messaage
_news_messages = []

### MRN Process Code

The MRN data can be subscribed with the *NewsTextAnalytics* domain and MRN-specific RIC name as following:
- *MRN_STORY*: Real-time News
- *MRN_TRNA*: News Analytics: Company and C&E assets
- *MRN_TRNA_DOC*: News Analytics: Macroeconomic News & events
- *MRN_TRSI*: News Sentiment Indices

### Initial Refresh Message
The Initial Refresh response does not contain any NTA data – but the relevant feed related or other static Fields are populated with appropriate values e.g. an MRN_STORY Refresh could contain: 

In [158]:
# MRN variables

mrn_domain = 'NewsTextAnalytics'
mrn_item = 'MRN_STORY'

def decodeFieldList(fieldList_dict):
    for key, value in fieldList_dict.items():
        print("Name = %s: Value = %s" % (key, value))


def send_mrn_request(ws):
    """ Create and send MRN request """
    mrn_req_json = {
        'ID': 2,
        "Domain": mrn_domain,
        'Key': {
            'Name': mrn_item
        }
    }

    ws.send(json.dumps(mrn_req_json))
    print("SENT:")
    print(json.dumps(mrn_req_json, sort_keys=True, indent=2, separators=(',', ':')))


def processRefresh(ws, message_json):

    print("RECEIVED: Refresh Message")
    decodeFieldList(message_json["Fields"])

def processStatus(ws, message_json):  # process incoming status message
    print("RECEIVED: Status Message")
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

### MRN News Update messages Process Code

The updates contain only fields related to the item and the fragment. They do not contain any of the static or per-feed fields. The updates are not cached or conflated.

#### First Update
The first update contains all the fields related to the item and the first fragment, subsequent updates only contain the fields relating to the fragment they contain. The FRAG_NUM FID is set to 1 for the first Update of each item and is incremented in each subsequent Update for that item. This allows you to you to detect a missing fragment (and ensure correct order of the fragments for re-assembly). 


#### Subsequent Update and Multi Fragment Items
The subsequent update contains the fields necessary to identify the MRN data item, the order of this fragment among all the fragments for this item, and the fragment itself. The other point to note is that (for a Multi fragment item), Update messages with FRAG_NUM >1 will have fewer FIDs as the metadata is included in the first Update message (FRAG_NUM=1) for that item

#### News Fragments simple handle logic


<img src="images/mrn_flowchart.png"/>


In [26]:
def processMRNUpdate(ws, message_json):  # process incoming News Update messages

    fields_data = message_json["Fields"]
    # Dump the FieldList first (for informational purposes)
    # decodeFieldList(message_json["Fields"])

    # declare variables
    tot_size = 0
    guid = None

    try:
        # Get data for all requried fields
        fragment = base64.b64decode(fields_data["FRAGMENT"])
        frag_num = int(fields_data["FRAG_NUM"])
        guid = fields_data["GUID"]
        mrn_src = fields_data["MRN_SRC"]

        #print("GUID  = %s" % guid)
        #print("FRAG_NUM = %d" % frag_num)
        #print("MRN_SRC = %s" % mrn_src)

        if frag_num > 1:  # We are now processing more than one part of an envelope - retrieve the current details
            guid_index = next((index for (index, d) in enumerate(
                _news_envelopes) if d["guid"] == guid), None)
            envelop = _news_envelopes[guid_index]
            if envelop and envelop["data"]["mrn_src"] == mrn_src and frag_num == envelop["data"]["frag_num"] + 1:
                print("process multiple fragments for guid %s" %
                      envelop["guid"])

                #print("fragment before merge = %d" % len(envelop["data"]["fragment"]))

                # Merge incoming data to existing news envelop and getting FRAGMENT and TOT_SIZE data to local variables
                fragment = envelop["data"]["fragment"] = envelop["data"]["fragment"] + fragment
                envelop["data"]["frag_num"] = frag_num
                tot_size = envelop["data"]["tot_size"]
                print("TOT_SIZE = %d" % tot_size)
                print("Current FRAGMENT length = %d" % len(fragment))

                # The multiple fragments news are not completed, waiting.
                if tot_size != len(fragment):
                    return None
                # The multiple fragments news are completed, delete assoiclate GUID envelop
                elif tot_size == len(fragment):
                    del _news_envelopes[guid_index]
            else:
                print("Error: Cannot find fragment for GUID %s with matching FRAG_NUM or MRN_SRC %s" % (
                    guid, mrn_src))
                return None
        else:  # FRAG_NUM = 1 The first fragment
            tot_size = int(fields_data["TOT_SIZE"])
            print("FRAGMENT length = %d" % len(fragment))
            # The fragment news is not completed, waiting and add this news data to envelop object.
            if tot_size != len(fragment):
                print("Add new fragments to news envelop for guid %s" % guid)
                _news_envelopes.append({  # the envelop object is a Python dictionary with GUID as a key and other fields are data
                    "guid": guid,
                    "data": {
                        "fragment": fragment,
                        "mrn_src": mrn_src,
                        "frag_num": frag_num,
                        "tot_size": tot_size
                    }
                })
                return None

        # News Fragment(s) completed, decompress and print data as JSON to console
        if tot_size == len(fragment):
            print("decompress News FRAGMENT(s) for GUID  %s" % guid)
            decompressed_data = zlib.decompress(fragment, zlib.MAX_WBITS | 32)
            
            json_news = json.loads(decompressed_data)
            _news_messages.append(json_news)
            print("News = %s" % json_news)

    except KeyError as keyerror:
        print('KeyError exception: ', keyerror)
    except IndexError as indexerror:
        print('IndexError exception: ', indexerror)
    except binascii.Error as b64error:
        print('base64 decoding exception:', b64error)
    except zlib.error as error:
        print('zlib decompressing exception: ', error)
    # Some console environments like Windows may encounter this unicode display as a limitation of OS
    except UnicodeEncodeError as encodeerror:
        print("UnicodeEncodeError exception. Cannot decode unicode character for %s in this enviroment: " %
              guid, encodeerror)
    except Exception as e:
        print('exception: ', sys.exc_info()[0])

### JSON-OMM Process functions

In [27]:
def process_message(ws, message_json):
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if "Domain" in message_json:
            message_domain = message_json["Domain"]
            if message_domain == "Login":
                process_login_response(ws, message_json)
            elif message_domain:
                processRefresh(ws, message_json)
    elif message_type == "Update":
        if "Domain" in message_json and message_json["Domain"] == mrn_domain:
            processMRNUpdate(ws, message_json)
    elif message_type == "Status":
        processStatus(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))


def process_login_response(ws, message_json):
    """ Send item request """
    send_mrn_request(ws)


def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': 1,
        "Domain": 'Login',
        'Key': {
            'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            }
        }
    }

    login_json['Key']['Name'] = user
    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

### WebSocket Process functions

In [28]:
def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Start websocket handshake
    ws_address = "ws://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(30)
    web_socket_app.close()

Connecting to WebSocket ws://172.20.33.30:15000/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "Position":"10.42.68.162"
    },
    "Name":"root"
  }
}
RECEIVED: 
[
  {
    "Domain":"Login",
    "Elements":{
      "MaxMsgSize":61430,
      "PingTimeout":30
    },
    "ID":1,
    "Key":{
      "Elements":{
        "AllowSuspectData":1,
        "ApplicationId":"256",
        "ApplicationName":"ADS",
        "Position":"10.42.68.162",
        "ProvidePermissionExpressions":1,
        "ProvidePermissionProfile":0,
        "SingleOpen":1,
        "SupportBatchRequests":7,
        "SupportEnhancedSymbolList":1,
        "SupportOMMPost":1,
        "SupportOptimizedPauseResume":1,
        "SupportPauseResume":1,
        "SupportStandby":1,
        "SupportViewRequests":1
      },
      "Name":"root"
    },
    "State":{
      "Data":"Ok",
      "Stream":"Open",
      "Text":"Login accepted by h

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2019-07-26",
      "FRAGMENT":"H4sIAAAAAAAC/52TTY/aMBCG7/0Vlq+Qkg82EN8CoSw0X02CWrVUK4cYNt2QsLaDFlX97x0nQT1Vqnrx+3g84xnb45+YVnJTYILrWDoPhRfO8RjTtihZfWACk284jEm8dvH3Mc6b4gaepr1tK8OZINMyED2jlZdNdBt4HWT7WtPuTFBcUSkF2tILrZHXnJmQ5QF9ppLxvOE1Qwm7NFyyAmWcFkygfR21UpQFQ80RBdESmTaCZMjUDQcW93XYQJRKMmy9rpqcVsitGJeatq+h+GPJhVxyBlnUuVSops800850m1gGmervdcf5Cp7PjBZVWTPwin03y1KCDEfXNFeUFM1n/1H/36uHdKUq537LT0+QambapvV2W5iFPmej0cFj1KW7rf7i0C/WdTRZcW9+Ch7Ozyq6FpKqN4mO8CrwGhWtTy09qeqZOjdUJ2Ca3S5gMmFenocJluxNTi4VLZXfhTdXqJGDPUxJ7GepMrZ5Kqls4cUx5JGkFTSvGKyINv/BDrJrBZdMwbIiRjdO9V6UbU0+whgQ45MHuiOm1cu0k4ddJ/ZjL34ns3knC7eTJYyJajS4K/3OPYYmcdON29PCXWY9LaNggDiJvB69TZr5fs+P0Wag7WoI8VdhD0G07iF04wGSwRIlXtLTn22TD5Hqf0lfWMpeW/U1MDHGuOUnYPgT1hhfGRdlU/9D4/169xuxyHs8dgMAAA==",
      "FRAG_NUM":1,
      "GUID":"Pt95dDN8__19072623xyB2d08e++cDeaAaUJ0k9aX3v+/ErD8gM5mh",


RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2019-07-26",
      "FRAGMENT":"H4sIAAAAAAAC/6VUW4/aRhR+768Y+aW7Ehfb7HKxSpUNkHRXsCEYFDWlrQb7gCfYM+5c2KAq/z3fAFl1pTxU7cgPnnO+853rnL8DXtr7PEgCOY0f45v30WoUNALuckEyIxMkvwWP8ySarFYp5Pi9f7N4E/zeCDYqP8KMTelAJcNZUlZIkfGSpWInxRa/YFhLFrWiOAqBGHNRHtkvYlewB1eyuHNWRoM+lFHYHPMjm919E8bPFkuSey6bKcmLrt//DtvtRXnThXI4XNU5t8TmWmQ0HF50UQ+6ztnTVD2RZq9VWQq5I31BhBEQcRgNvP4lcehzmJtjVqhS7U6ZnpI/q8Pe7XNQ3nQGF1HfU/UASK1GNLsje3He1STZXBlhhfK5jZzWJO2z6IxKC6Ute+VDuA2BWnK9I5ucnMbhN67Uqvoki2MP8tTtUakM/dNhfNtESs1oAMRktWiv0vGPKat9jRjPTi7xLQunTY74DRrJS8OEzCkTBuqEPYmcmOYoGX5twTjLvJe1lMQ1swUxBdctkBBzsNPlEfVlG2iFKZix2mXWaXBQxYU0DVagEQfSDVYRlx4LkrVUEPGyZLDaM2FgKHBDdN5Frp6k8YFslWYwltahG4jDkAFAK4epeNGBS4dK9XQOTdKOW3FA2iWSrHzV1fbEbU/TxrjM2V58cpJhPsDK8xzU6hzcdxLbCG5a7Go2u177VuL89Md/P2t5nqT7rBCV2js2KrhGz/0prK1N0m7bykhtW9q041ldjcPl2e3/8PozGK6uKngSslWh3qRf2UJVRklNzpI2rUxV19fYBF

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2019-07-26",
      "FRAGMENT":"H4sIAAAAAAAC/42TS3PbIBDH7/0UDIeeHEcPyw9mMh1VkR9pjBzhKI+m00EytqllpAqUNO30uxckOe2xF/4/lt2FXeAXpLlabCCCYh4Fk5n3nE5hD9J6w5nImIToM8QrlCyCCH7pwbTYvGrfaVGBlEkFKibrXEnwsmcClBUXiosdUHsuARWiqHWKIxOqB8qcUclAlvPsAAoBci4OOkVevKAnsVeqROfn5Wbbr1itWCX7WXE0c8Fe5En7VJYf+MXAtbxJ5qZbi7kjb2C/ry/qSiDjcczRP/HIseyJNXKGyJSW/rxX0+mTeBK6vC2vpAoqRhUzpRvHM2t05gzX1hC5Nhp4/fHIedSee0Y3+qxMe7XJAE7WQPKdMBUCX/epKTkohKpopsAZuME6jpu8bx392sY6g2Q7w7YvZXk758khurpdX87o7Djc7WlekKtDLIhrooVU1LQ/2uoL0H3PqdjVdGeOwUwBRyalnq5fS21y9JwfuwlU7Ic6L3PKjV9ZFc98wyptxwTNIxIaY50SRVWtLxfqfRSqJU1zpldknX5jmWpu3Ue2tnxE9sByGnA8r9HhpJHxsJVRK2MtsyZkhpzwU6PeYyPBopHWhpuR6HGJ7JvLVu+uOr3uFDfqOq3MGzHJYqTb358vNWIH+ST08QkX/onuV37QchBEpKPlCj90GOEVeUOSdPgQkL9kjVsOl/GsI7zu1hf4MpgvWr4Ou/1xeBe3FIddREyi+BSjD/p2QBInp63I2n840X0LiUm4Qp5l6RfjuQPXfDtFD4yw77X5khDZPVhXO836K7o9+KzfOy

RECEIVED: 
[
  {
    "Type":"Ping"
  }
]
SENT:
{
  "Type":"Pong"
}
RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2019-07-26",
      "FRAGMENT":"H4sIAAAAAAAC/6VY7ZIitxX9n6dQUevybgUYvgZmsGvLDMvMsDMwGPB414vjEt2C1tJI7ZZ6WJJKVV4jr5cnyblSd8NWnD8JP5BQS7pX5557dJu/VXhsx2GlX1GPnWmrM2/OlpVqhWehFCoQptL/VJnO+kOMoXnnm5FvJr558s3SNz/lg+NH35kOigcPY9+5m+crJst8zuLp1ncGt1PfeZ8/yMd/8c1sMaz8Wq2sdXiEw+z1MhKMZzbSKZOGcTYXmRWpYTep4Dupti9SHAwLdJztlTS2zmiBTqSSWpmVEl+SVBgjQsZTwSKRMn1Q9TcrxfC5ObK5Xh8Vm/CdH7l/mt6xB3xV2fssPrJWl73+Q4tvWI1NpbJChfpb2Pm+d93r1Jdv2ToLQ0xiPI4lB7rsIG3ElgS0suz7Rq/RqN8/vIXHWRyyPU93zEZipdZiK5WilXqDY64FDi03Wcw2KaIUmkgm/mz2oNmLDIWubfleGLaFGQsP6HxbwVPaIkuY1SzmmQoi2p695wlXwmBGqrPkW8MWcAvPAoCkY8GkWqlhJBWvs1mqTSICa8iRUIgEmFkJQ7HWOyZTwAmc5ToW/RIBFnBsYHlSHPRf//inYXu9ltgbIRCplTDOVcgyg/3W3IgqO0T0WMKSsRz7rRQsBvpFWIQriHjKAwd8LHcCIUqldjs8ZnIrySJba21sYZKisI31mscMgQqiuo+ow+xVp8PgSwxOMJ3ZjbQAGMcNHTj

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2019-07-26",
      "FRAGMENT":"H4sIAAAAAAAC/7VWW28bRRR+51dY++rsdG47lxUgOa5V4sSJ06RQQRHaNNvgkDppvI4SIaSmSNCiAg9QKhBFqgQVT7QgKiiN82Ma4vbNf6FnZmfW61AkhGAfds6enfnO7Tsz836QbGQzq0EcdJvNJsvWXltPgqkg6a920u7FtBfEbwXz7bhZb4AWhLPN+bqTlurB21PByubqHqyuVJ5cfby8UF9oDW9+PLz1YHjv2rN7+0f7919ut7l8dTT4fnT4zejwk9Hhd0YY3B8NDowM+sEvo8M77tfgwWgw+PP3uznI0f5Pzx7+9vTaI8DB+vjRr4QOv73O8PD2D68QDA/CuGKeC91K6Sm5YT6Pb3wKC4b3v6xUXuhDMbM0sfJXr1447aTp/+K50CUaSyr8N5URktqKVYxU9G8Q/ycfo7GPHGluxRAjkXtbkVIpJE2JQqYRcVoRMYIiacIhAjEygcjLUQuWR00RYw5RUYmUNksBhDKPSCJkExZGAmE6gcjGiAwx7H3kzq4UWiBh11KGSIEoAV5Z6xopNoFIy4gyx6kSpJWPmmhbsCoh2DqbIwqCuElYyBUioozoM2MQCZKs8FF6H7FC2uaRQsacVgiskSDWOEFMTyCqMSJF1EVNCvZIwSBj0ugwRxH2PmqFbHhVzpGvhUOU5VrjHCfkhTdSQomlQaQRRM09YsRsYqoE+4QWiCWGK8TzrIYUKZ9HrbGNLyRCIuwReURzTlVFdDLqcWUUKdWaOUuKRBIJWwOKkXYMEExJR8MqVCaaQCRlRJHjVLmXABFcj4ThqBDIM0MwqW2JIb

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2019-07-26",
      "FRAGMENT":"H4sIAAAAAAAC/3WTTXObMBCG7/0VGl16cRKEA9iaTmcwyIlsLBzER+NOp4NjOaV1cGogbabT/94VKMmpFz3vvuxq0dcfXB5avsMU12wiSLG0bIJHuOx2larvVIPpZyzWlIEHyJYcfxnh7XH3DBXg7atT0wYnVbZKz2FbZHpmeWe2m1outafUuTy3LGsDmd9UuTtUtYIsufa5eC+R9EXqi5Al6API81XwEaxbiVA8n4MZsTQFrPyQoTRGPPUj7gs088USbChMmI/iJGAR4hKJOEU+CmKRJn4AUqKE3WQ8YSGa3SJoKbi8RpFfjFABHwXLYZIgXq2hDeTECZpn0ZxHEQTwu5VezuuOfCVTy7NdssiXzk66mwvLIny/7eSnssjDriBRthfEsa7jX2NPV9dNW+rti/ewgbBjh7K+78p7vXpVQ8KDahoI0+dHsAjE1YMJcKt+txePh7LSeY+n41O1UyfwhaRJmkhtdlvZlm0Hh4OhT0u7ptweFHxpuu13ddf2p+ZTG5wZJbZj6Bp6hpOBLhkSYH2GfWFEhxioY0ZJP+qbcEXH/eg4PbxFD38YI8CKkptwYCF6Mm+AHJD30PciocPZ6wtm0xmc3YsSS6O4zN+UNTH69bNYyjdFjJS5mTGIFkas1uJ2kOHMNGGmkmVJvDZyYwrnXLwK+aZe+l/NDJlJW2QhN6XRi7cSV1yaYpnkgZEFy0Cs6cSZTKfjS9ty9Ktqyx9Kqp+dfneYjke4O92Dfu4vyJM6NdWx/t9TGxPqeOfE9Tb477t/WT6FJNQDAAA=",
      "FRAG_NUM

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2019-07-26",
      "FRAGMENT":"H4sIAAAAAAAC/41V227jNhB971cQQtELIDuSs3XWRtE2m+s2ieOuvFigVrCgLcrihiJVXux1i/57z0hykkX7UBuwJZIzc+bMGc5fEVf+bRFNIz336Z9XfraP4oiHQgq9Fi6aLqPZfJpdz0+jhzhamWKPo1fBbrjR7Lu54t677weD0fjXoEZJOjkaHaeM1+zifHE0xuPV3SLX9H0T9KOwrBA11wWTml0bvWE39CPrxpqtKJgSvJBY8IZVclPh+KqzaqwEGHZpZcH3ca5rbh+FZ84ES+uOy2LI8ui8d+6Yh7EYIsZObIWNu3fa0IbxLZfKDZ8B5BobOLZnHlE9s+2vNrsYEcjorJKaM3KdAR5vDHwtV8E/HPCtud0Y5ioTVJFr57n1bG1qygWZChiakoGhfesEYFh3BqunYRP3q9z3LpgVSgI4XJla0CkyWa6DtUIjagtTC+fyCLYHEN7yAn8HMs5aTI0VtQy1A45cP6Ov+FYwXpZi7cH79Q3bWME98JXGEqTgPNx6tjzYP7Cd0d+CG+GD1ahQrrWxNVcAoxTCmBVfUX6lB4Y08RVbdn4eOpDP5XahaZDeAemiQp59mDb808lBARpQFyA8fp2ws2xxSLUMQjFIxrbE3AmuiaROjbkm7SCKKsPhpFSMOwfCavDnkETNpYZbvoKT1sfXaZIOT0ZHtYcKt7BxEvrPdVu0dBKTJlei90KWHiZpMhyNyQS9sKiCdaROln0zZ1cKfKgeESu456hlBTUiYYK7q6QSpI3MBFB1g5rwuM+OVNO3ScUdogrdt0ivpwbSyXVpTc2U2R

WebSocket Closed


### Real-time News Data Model
The news data appears as JSON in UTF-8 after decompression and assembly of the individual messages.
The Real-time News feed contains the headline, story body text, and associated metadata about the story as a simple group of named values.

Example JSON fields are following:
- ```firstCreated```: UTC timestamp for the first version of the story. Millisecond precision.
- ```headline```: The headline text of the news item.
- ```id```: Uniquely identifies the news item. This is the same value as the GUID in the OMM envelope.
- ```body```: The full body text of the news item (Story Body).

For more detail regarding MRN data model please visit [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item) page. If you want to compare MRN and the legacy N2_UBMS news data model, please visit [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).



In [156]:
print("first 10 headlines\n")
for news in _news_messages[:10]:
    if news["headline"]:
        print(news["headline"])

first 10 headlines
Change in substantial holding from PPT,PPT-ORA.AX
PLATTS: 190--Asia 86: Platts Japan Waterborne MOC Deals Summary 26 Jul 2019
◎ﾆｭｰﾖｰｸ･ﾏｰｶﾝﾀｲﾙ原油先物時間外取引      (USﾄﾞﾙ/ﾊﾞﾚﾙ､時刻=JST､10分ﾃﾞｨﾚｰ) -ﾛｲﾀｰ- GM96 15:31
PLATTS: 190--Asia 87: Platts Japan Domestic Waterborne Reported Trades Outside of MOC 26 Jul 2019
[環球匯市] 市場速遞：丹斯克：從基本面看應逢低買入EUR/USD
[环球汇市] 市场速递：丹斯克：从基本面看应逢低买入EUR/USD
EUR/USD Trader
190726 Change of personnel -QN
190726 The record date for the dividend payment - QN
190726 NVT signs an Auditing Contract - QN


In [157]:
print("first 10 stories\n")
for news in _news_messages[:10]:
    if news["body"]:
        print(news["body"])

first 10 stories
http://pdf.reuters.com/asxnews/asxnews.asp?i=43059c3bf0e37541&u=urn:newsml:reuters.com:20190726_nASX7JGnBx

26Jul19/ 231 am EDT/0631 GMT
--0631 GMT: Platts Japan Waterborne MOC Deals Summary 26 Jul 
2019 

None 
--Platts Global Alert--

190726                                                             190725 
15:31 現在値 前日比       買気配 売気配 高値   安値   始値   VOL   ｾﾂﾙﾒﾝﾄ 取組高
1909  56.25  +0.23  15:21 56.24  56.25  56.34  55.85  55.93  36281 56.02  419054
1910  56.37  +0.21  15:21 56.37  56.38  56.46  55.97  56.05  4047  56.16  182011
1911  56.47  +0.21  15:19 56.46  56.48  56.55  56.11  56.12  1726  56.26  137472
1912  56.48  +0.21  15:17 56.46  56.48  56.53  56.08  56.15  1157  56.27  294441
2001  56.45  +0.25  14:32 56.38  56.39  56.45  56.11  56.13  823   56.20  119981
2002  56.27  +0.23  14:39 56.22  56.24  56.30  56.02  56.02  212   56.04  53015 
2003  55.97  +0.13  14:01 56.01  56.03  55.97  55.78  55.78  101   55.84  57091 
2004  ---                 55.77  55.79  --- 

In [38]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install pandas and numpy packages in a current Jupyter kernal\n

# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install numpy

In [149]:
import pandas as pd

In [150]:
df_headlines = pd.DataFrame(_news_messages, columns = ["firstCreated","messageType","provider","language","headline"])

In [151]:
messageType = {
    0: "Unknow",
    1: "Alert",
    2: "First take",
    3: "Subsequent take",
    4: "Correction",
    5: "Corrected",
    6: "Update",
    7: "Deletion",
    8: "Drop due to expiry"
}

In [152]:
df_headlines["messageType"] = [messageType[value] for value in df_headlines["messageType"] if value in messageType ]

In [153]:
df_headlines

,firstCreated,messageType,provider,language,headline
0,2019-07-26T06:10:45.977Z,First take,NS:ASX,en,"Change in substantial holding from PPT,PPT-ORA.AX"
1,2019-07-26T06:31:34.232Z,First take,NS:PLTS,en,PLATTS: 190--Asia 86: Platts Japan Waterborne ...
2,2019-07-25T22:31:41.000Z,First take,NS:JIJI,ja,◎ﾆｭｰﾖｰｸ･ﾏｰｶﾝﾀｲﾙ原油先物時間外取引 (USﾄﾞﾙ/ﾊﾞﾚﾙ､時刻=J...
3,2019-07-26T06:31:40.099Z,First take,NS:PLTS,en,PLATTS: 190--Asia 87: Platts Japan Domestic Wa...
4,2019-07-26T06:31:42.010Z,First take,NS:IGM,zh,[環球匯市] 市場速遞：丹斯克：從基本面看應逢低買入EUR/USD
5,2019-07-26T06:31:42.021Z,First take,NS:IGM,zh,[环球汇市] 市场速递：丹斯克：从基本面看应逢低买入EUR/USD
6,2019-07-26T06:31:42.000Z,First take,NS:RTRS,en,EUR/USD Trader
7,2019-07-26T06:31:44.904Z,First take,NS:HOSE,en,190726 Change of personnel -QN
8,2019-07-26T06:31:45.394Z,First take,NS:HOSE,en,190726 The record date for the dividend paymen...
9,2019-07-26T06:31:45.872Z,First take,NS:HOSE,en,190726 NVT signs an Auditing Contract - QN


In [154]:
df_story = pd.DataFrame(_news_messages, columns = ["headline","body"])

In [155]:
df_story

,headline,body
0,"Change in substantial holding from PPT,PPT-ORA.AX",http://pdf.reuters.com/asxnews/asxnews.asp?i=4...
1,PLATTS: 190--Asia 86: Platts Japan Waterborne ...,26Jul19/ 231 am EDT/0631 GMT\n--0631 GMT: Plat...
2,◎ﾆｭｰﾖｰｸ･ﾏｰｶﾝﾀｲﾙ原油先物時間外取引 (USﾄﾞﾙ/ﾊﾞﾚﾙ､時刻=J...,190726 ...
3,PLATTS: 190--Asia 87: Platts Japan Domestic Wa...,26Jul19/ 231 am EDT/0631 GMT\n--0631 GMT: Plat...
4,[環球匯市] 市場速遞：丹斯克：從基本面看應逢低買入EUR/USD,\n（北京時間2019年07月26日14:31）[IGM Global MarketWatc...
5,[环球汇市] 市场速递：丹斯克：从基本面看应逢低买入EUR/USD,\n（北京时间2019年07月26日14:31）[IGM Global MarketWatc...
6,EUR/USD Trader,Level Technical Significance\n 1.1210 Da...
7,190726 Change of personnel -QN,For best results when printing this announceme...
8,190726 The record date for the dividend paymen...,For best results when printing this announceme...
9,190726 NVT signs an Auditing Contract - QN,For best results when printing this announceme...


## References
* [Refinitiv Elektron SDK Family page](https://developers.refinitiv.com/elektron) on the [Refinitiv Developer Community](https://developers.thomsonreuters.com/) web site.
* [Refinitiv Elektron WebSocket API page](https://developers.refinitiv.com/websocket-api).
* [Developer Webinar Recording: Introduction to Electron WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
* [Introduction to Machine Readable News with Elektron WebSocket API](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor](https://github.com/Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor).

For any question related to this example or Elektron WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/152/websocket-api.html).